In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Load data
df = pd.read_csv("ranchi_df.csv")
df = df.dropna(subset=["Price (in lakhs)"])

# Log-transform the target to reduce skew
y = np.log1p(df["Price (in lakhs)"])
X = df.drop(columns=["Price (in lakhs)", "Index"])

# Feature groups
numerical_cols = ["Floor (Ratio)", "Bathroom", "Balcony", "BHK", "Final Carpet Area (in sqft)"]
categorical_cols = ["Furnishing", "facing", "Ownership", "Car Parking", "Transaction", "Garden/Park", "Main Road", "Pool"]

# Preprocessing pipelines
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, numerical_cols),
    ('cat', cat_pipeline, categorical_cols)
])

# Initial model pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

# Extract important features
model = pipeline.named_steps['model']
ohe = pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot']
ohe_feature_names = ohe.get_feature_names_out(categorical_cols)
feature_names = numerical_cols + list(ohe_feature_names)
importances = model.feature_importances_

# Filter important features (threshold: > 0.01)
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
important_features = importance_df[importance_df['importance'] > 0.00]['feature'].tolist()

# Map back to original column names
important_numerical = [col for col in numerical_cols if col in important_features]
important_categorical = [col for col in categorical_cols if any(col in f for f in important_features)]

print("Selected Numerical Columns:", important_numerical)
print("Selected Categorical Columns:", important_categorical)

# Redefine preprocessors with reduced columns
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, important_numerical),
    ('cat', cat_pipeline, important_categorical)
])

# Final model pipeline
final_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train on reduced features
X_train = X_train[important_numerical + important_categorical]
X_test = X_test[important_numerical + important_categorical]
final_pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred_log = final_pipeline.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_actual = np.expm1(y_test)

rmse = np.sqrt(mean_squared_error(y_actual, y_pred))
r2 = r2_score(y_actual, y_pred)
relative_rmse = rmse / y_actual.mean()

print(f"RMSE: {rmse:.2f}")
print(f"Relative RMSE: {relative_rmse:.2f}")
print(f"R² Score: {r2:.2f}")


Selected Numerical Columns: ['Floor (Ratio)', 'Bathroom', 'Balcony', 'BHK', 'Final Carpet Area (in sqft)']
Selected Categorical Columns: ['Furnishing', 'facing', 'Ownership', 'Car Parking', 'Transaction', 'Garden/Park', 'Main Road', 'Pool']
RMSE: 13.93
Relative RMSE: 0.24
R² Score: 0.68


In [3]:
import os
import joblib

# Create folder if it doesn't exist
os.makedirs("website/models", exist_ok=True)

# Save the full pipeline — not just the model
joblib.dump(final_pipeline, "website/models/ranchi_model.pkl")

['website/models/ranchi_model.pkl']

In [4]:
# --- INFERENCE ---
import joblib
import pandas as pd

# Load the full pipeline
pipeline = joblib.load("website/models/ranchi_model.pkl")

# Extract preprocessor to get used columns
preprocessor = pipeline.named_steps["preprocessor"]
used_num_cols = preprocessor.transformers_[0][2]
used_cat_cols = preprocessor.transformers_[1][2]
used_columns = used_num_cols + used_cat_cols

# Full input from user (can include unused features — that's okay)
input_data = {
    "Floor (Ratio)": 0.5,
    "Bathroom": 2,
    "Balcony": 1,
    "Car Parking": 1,
    "BHK": 2,
    "Final Carpet Area (in sqft)": 950,
    "Furnishing": "Furnished",
    "facing": "East",
    "Ownership": "Freehold",
    "Transaction": "Resale",
    "Garden/Park": 1,
    "Main Road": 1,
    "Pool": 0,
    "Not Available": 0  # Extra column, will be ignored
}

input_df = pd.DataFrame([input_data])
input_df_filtered = input_df[used_columns]

# Predict
log_pred = pipeline.predict(input_df_filtered)[0]
predicted_price = np.expm1(log_pred)  # Correct way to invert log1p
print(f"Predicted Price: ₹{predicted_price:.2f} lakhs")

Predicted Price: ₹50.07 lakhs


In [5]:
df


,Index,Price (in lakhs),City,Floor (Ratio),Furnishing,facing,Bathroom,Balcony,Car Parking,Ownership,BHK,Location,Final Carpet Area (in sqft),CurrentFloor,Transaction,Garden/Park,Main Road,Not Available,Pool
0,0,45.4,ranchi,0.555556,0,-1,2,0,-1,-1,2.0,Sector 2 Masibari,920.0,5.0,1,0,0,0,0
1,1,57.2,ranchi,0.888889,0,90,2,2,1,0,3.0,Argora,1040.0,8.0,1,1,1,0,0
2,2,78.0,ranchi,0.111111,0,90,2,1,1,0,3.0,Argora,1380.0,1.0,1,0,1,0,0
3,3,60.0,ranchi,0.750000,2,-1,3,0,-1,-1,3.0,Shalimar Bagh Kathal More,1373.6,3.0,0,0,0,0,0
4,4,38.0,ranchi,0.750000,1,45,2,2,1,0,2.0,Namkum,784.0,3.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,1075,54.0,ranchi,0.666667,1,-1,3,2,-1,-1,3.0,Shalimar Bagh Kathal More,1440.0,4.0,0,0,0,0,0
1076,1076,189.0,ranchi,0.250000,1,90,3,3,-1,0,3.0,Morabadi,1685.6,2.0,0,1,1,0,0
1077,1077,123.0,ranchi,0.875000,0,135,4,4,-1,0,3.0,Morabadi,1433.0,7.0,0,0,1,0,0
1078,1078,45.8,ranchi,0.500000,0,90,2,2,1,0,2.0,Bariatu,1077.6,2.0,1,0,1,0,0
